In [9]:
using JuMP
using Ipopt

NS=2
S=1:NS
rhs=zeros(NS)
pr=zeros(NS)
rhs[1]=500
rhs[2]=700
pr[1]=0.6
pr[2]=0.4

# dual subproblems
function solve_subproblem(s,rhs,pr,lambda) 
    m=Model(solver=IpoptSolver(print_level=0));
    @variable(m,x>=0)
    @variable(m,y>=0)
    @constraint(m,cons, x+y>=rhs[s])
    @objective(m,Min,pr[s]*(2*x+3*y) + lambda[s]*x)
    solve(m)   
    return getvalue(x),getobjectivevalue(m)
end

# compute upper bound
function solve_upperbound(x,rhs,pr,S)
    m=Model(solver=IpoptSolver(print_level=0));
    @variable(m,y[S]>=0)
    @constraint(m,cons[s in S], x+y[s]>=rhs[s])
    @objective(m,Min,sum(pr[s]*(2*x+3*y[s]) for  s in S))
    solve(m)  
    return(getobjectivevalue(m))
end

Nit=3       # number of iterations
lam=0       # initial dual variable
x=zeros(NS) # container for first-stage
D=zeros(NS) # container for dual function values
lambda=zeros(NS) # constainer for duals
alpha=0.001 # step-size duals
zUB=0;      # Upper bound 

for it=1:Nit
    
    # get lower bound
    lambda[1]=-lam
    lambda[2]=+lam
    for s=1:NS
        (x[s],D[s])=solve_subproblem(s,rhs,pr,lambda)    
    end
    println('\n')
    @printf("lambda=%e\n", lam)    
    println(x)
    println(D)
    err=x[1]-x[2]
    @printf("Error=%e\n", err)
    zLB=sum(D)
    @printf("zLB=%e\n", zLB)
    
    # get upper bound for every x[s]
    for s=1:NS
        zUB=solve_upperbound(x[s],rhs,pr,S)    
        gap=zUB-zLB
        @printf("zUB=%e\n", zUB)
        @printf("Gap=%e\n", gap)
    end
    
    # update duals
    lam=lam-alpha*(err)
end



lambda=0.000000e+00
[500.0,700.0]
[600.0,560.0]
Error=-2.000000e+02
zLB=1.160000e+03
zUB=1.240000e+03
Gap=8.000001e+01
zUB=1.400000e+03
Gap=2.400000e+02


lambda=2.000000e-01
[500.0,700.0]
[500.0,700.0]
Error=-2.000000e+02
zLB=1.200000e+03
zUB=1.240000e+03
Gap=4.000001e+01
zUB=1.400000e+03
Gap=2.000000e+02


lambda=4.000000e-01
[500.0,700.0]
[400.0,840.0]
Error=-2.000000e+02
zLB=1.240000e+03
zUB=1.240000e+03
Gap=1.578096e-05
zUB=1.400000e+03
Gap=1.600000e+02
